In [41]:
!pip install datasets transformers
!pip install evaluate
!pip install rouge_score
!pip install -U bitsandbytes
!pip install faiss-cpu 
!pip install langchain
!pip install langchain_community

In [42]:
from datasets import load_dataset
from transformers import PreTrainedTokenizerBase
from transformers import AutoTokenizer, DPRContextEncoder,DPRContextEncoderTokenizerFast, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer, pipeline
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration, pipeline, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from datasets import DatasetDict
import evaluate
import torch
import faiss
from datasets import Dataset
from langchain.schema    import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings   import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms         import HuggingFacePipeline
from langchain.chains       import RetrievalQA
from langchain.prompts import PromptTemplate
import math
import re

## Helper functions

In [44]:
def extract_ans(raw_str: str) -> str:
    m = re.search(r"<ans>(.*?)</ans>", raw_str, flags=re.S)
    return m.group(1).strip() if m else raw_str.strip()

In [45]:
def preprocess(examples, tokenizer: PreTrainedTokenizerBase):
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=256,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for i, offsets in enumerate(offset_mapping):
        sample_idx = sample_mapping[i]
        answers = examples["answers"][sample_idx]
        seq_ids = tokenized.sequence_ids(i)
        cls_index = 0
        start_positions.append(cls_index)
        end_positions.append(cls_index)
        if len(answers["answer_start"]) == 0:
            continue
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0])
        context_tokens = [idx for idx, s_id in enumerate(seq_ids) if s_id == 1]
        if not context_tokens:
            continue
        chunk_start_char = offsets[context_tokens[0]][0]
        chunk_end_char = offsets[context_tokens[-1]][1]
        if not (chunk_start_char <= start_char and end_char <= chunk_end_char):
            continue
        token_start_index = cls_index
        token_end_index = cls_index
        for idx in context_tokens:
            off_start, off_end = offsets[idx]
            if off_start <= start_char < off_end:
                token_start_index = idx
            if off_start < end_char <= off_end:
                token_end_index = idx
        start_positions[-1] = token_start_index
        end_positions[-1] = token_end_index
    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized


In [46]:
def print_metrics(ds, custom_pipeline):
    outputs = custom_pipeline(question=ds["question"], context=ds["context"])
    
    preds = [
        {"id": ex["id"], "prediction_text": out["answer"]}
        for ex, out in zip(ds, outputs)
    ]
    refs = [
        {"id": ex["id"], "answers": ex["answers"]}
        for ex in ds
    ]
    
    results = metric.compute(predictions=preds, references=refs)
    print(f"EM: {results['exact_match']:.2f}, F1: {results['f1']:.2f}")

## Dataset loading

In [47]:
raw = load_dataset("squad")
sub = raw["train"].shuffle(seed=42).select(range(20000))
split = sub.train_test_split(test_size=0.2, seed=42)

test_subset = raw["validation"].shuffle(seed=42).select(range(2_000))
data = DatasetDict({
    "train":      split["train"],
    "validation": split["test"],
    "test":       test_subset
})

## Experiment 1

In [ ]:
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model_distilbert  = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
tokenized = data.map(
    lambda ex: preprocess(ex, tokenizer),
    batched=True,
    remove_columns=raw["train"].column_names
)

In [ ]:
args = TrainingArguments(
  output_dir="distilbert-qa",
  per_device_train_batch_size=128,
  per_device_eval_batch_size=128,
  num_train_epochs=10,
  learning_rate=3e-5,
  weight_decay=0.01,
  logging_steps=200,
  log_level="info",
  report_to=["none"],       
  disable_tqdm=False,      
  fp16=True if torch.cuda.is_available() else False,            
)

In [ ]:
trainer_full = Trainer(
  model=model_distilbert,
  args=args,
  train_dataset=tokenized["train"],
  eval_dataset =tokenized["validation"],
  processing_class=tokenizer,
  compute_metrics=compute_metrics
)
trainer_full.train()

In [ ]:
metric = evaluate.load("squad")
squad  = evaluate.load("squad")
rouge  = evaluate.load("rouge")
bleu   = evaluate.load("bleu")
qa = pipeline(
    "question-answering",
    model="/kaggle/working/distilbert-qa/checkpoint-1740",
    tokenizer="/kaggle/working/distilbert-qa/checkpoint-1740",
    device=0,
)

ds = data["test"]

print_metrics(ds, qa)

## Experiment 2

In [ ]:
model_partial = AutoModelForQuestionAnswering.from_pretrained(model_name)

#### Partial fine tuning

In [ ]:
for param in model_partial.distilbert.parameters():
    param.requires_grad = False


In [ ]:
trainer_partial = Trainer(
    model=model_partial,
    args=args_partial,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
)
trainer_partial.train()

In [ ]:
qa_partial = pipeline(
    "question-answering",
    model="distilbert-qa-partial/checkpoint-417",  
    tokenizer="distilbert-qa-partial/checkpoint-417",
    device=0 if torch.cuda.is_available() else -1,
)


print_metrics(data["test"], qa_partial)

## Experiment 3

In [48]:
contexts = raw["train"]["context"] + raw["validation"]["context"]
unique   = list(dict.fromkeys(contexts))
docs     = [Document(page_content=c) for c in unique]


#we need smaller pieces to fit the model 
#also it improves semantic search accuracy as smaller chunks allow the faiss index to capture a more specific meaning 
#chunk overlap preserves context
splitter    = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs  = splitter.split_documents(docs)


#Building an embedding for each chunk
embeddings = HuggingFaceEmbeddings() 
#Where we store the chunk and the corresponding embedding
#Faiss index contains vectors 
db         = FAISS.from_documents(split_docs, embeddings)
#What will help us retrieve the 4 most relevant using the index inialized in the previous line
retriever  = db.as_retriever(search_kwargs={"k": 4})

/tmp/ipykernel_35/3822853403.py:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [49]:


MODEL_ID = "google/flan-t5-large"          # swap to any row above

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_ID,
    device_map="auto",                  
                     
    # torch_dtype=torch.float16           # fp16 on GPU chunks
)

qa_pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    max_new_tokens=64,
)

llm = HuggingFacePipeline(pipeline=qa_pipe)

Device set to use cuda:0


In [50]:
zero_shot_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "{context}\n\n"
        "Question: {question}\n"
        "Answer:"
    ),
)

zero_shot_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": zero_shot_prompt},
)

In [51]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

cot_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Use the following passages to answer the question.\n"
        "First think step-by-step, then give the final answer on a new line "
        "wrapped in <ans>…</ans>.\n\n"
        "Passages:\n{context}\n\n"
        "Question: {question}\n\n"
        "Step-by-step reasoning:\n"
        "1. "
    ),
)

cot_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",          # or "map_reduce" if you prefer
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": cot_prompt},
)


In [52]:
def print_chain_metrics(chain, data,  apply_extract: bool = True):
    preds, refs = [], []
    rouge_hyp, rouge_ref, bleu_refs = [], [], []

    total = len(data["test"])
    for idx, ex in enumerate(data["test"], 1):
        print(f"[{idx}/{total}] Processing QID={ex['id']}…")

        # ↘ call the chain with a dict and extract the answer text
        raw = chain({"query": ex["question"]})["result"]
        text = extract_ans(raw) if apply_extract else raw.strip()               # <── NEW

        preds.append({"id": ex["id"], "prediction_text": text})
        refs.append({
            "id": ex["id"],
            "answers": {
                "text": ex["answers"]["text"],
                "answer_start": ex["answers"]["answer_start"]
            }
        })
        rouge_hyp.append(text)
        rouge_ref.append(ex["answers"]["text"][0])
        bleu_refs.append(ex["answers"]["text"])

        print(f"[{idx}/{total}] Done.\n")

    print("All examples processed — now computing metrics…\n")

    s_res = squad.compute(predictions=preds, references=refs)
    r_res = rouge.compute(predictions=rouge_hyp, references=rouge_ref)
    b_res = bleu.compute(predictions=rouge_hyp, references=bleu_refs)

    print(f"EM:           {s_res['exact_match']:.2f}")
    print(f"F1:           {s_res['f1']:.2f}")
    print(f"ROUGE-L F1:   {r_res['rougeL'] * 100:.2f}")
    print(f"BLEU:         {b_res['bleu'] * 100:.2f}")


In [ ]:
squad  = evaluate.load("squad")
rouge  = evaluate.load("rouge")
bleu   = evaluate.load("bleu")
print_chain_metrics(cot_chain, data,False)

Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[1/2000] Processing QID=572759665951b619008f8884…


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[1/2000] Done.

[2/2000] Processing QID=57296de03f37b3190047839e…


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

[60/2000] Done.

[61/2000] Processing QID=57332442d058e614000b5723…


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[61/2000] Done.

[62/2000] Processing QID=572906e23f37b31900477f8e…


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[62/2000] Done.

[63/2000] Processing QID=5728455bff5b5019007da078…


In [ ]:
squad  = evaluate.load("squad")
rouge  = evaluate.load("rouge")
bleu   = evaluate.load("bleu")
print_chain_metrics(zero_shot_chain, data, apply_extract=False)


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[1/2000] Processing QID=572759665951b619008f8884…


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[1/2000] Done.

[2/2000] Processing QID=57296de03f37b3190047839e…
[2/2000] Done.

[3/2000] Processing QID=5726d4a45951b619008f7f6c…


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[523/2000] Done.

[524/2000] Processing QID=572634a789a1e219009ac56f…


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[524/2000] Done.

[525/2000] Processing QID=57269d745951b619008f77d9…
[525/2000] Done.

[526/2000] Processing QID=573005b9947a6a140053cf6e…


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [91]:
# N = 5   # how many examples to inspect

# for i in range(N):
#     ex       = data["test"][i]           # one SQuAD example
#     question = ex["question"]

#     # ❱❱  feed the question as {"query": ...}
#     res = cot_chain({"query": question})   # or {"question": question} if you changed input_key
#     raw = res["result"]                    # chain output (reasoning + answer)
#     final = raw.split("Final Answer:")[-1].strip()

#     print(f"\n=== Example {i+1}/{N} ===")
#     print("QUESTION:", question)
#     print("RAW COT OUTPUT:\n", raw)
#     print("EXTRACTED ANSWER:", final)
#     print("GOLD ANSWERS:", ex["answers"]["text"])

Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 1/5 ===
QUESTION: In what year did Massachusetts first require children to be educated in schools?
RAW COT OUTPUT:
 1852. So the final answer is 1852.
EXTRACTED ANSWER: 1852. So the final answer is 1852.
GOLD ANSWERS: ['1852', '1852', '1852']


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 2/5 ===
QUESTION: When were stromules discovered?
RAW COT OUTPUT:
 The first observed in 1962. 2. In 1665. 3. In living plant tissue.
EXTRACTED ANSWER: The first observed in 1962. 2. In 1665. 3. In living plant tissue.
GOLD ANSWERS: ['1962', '1962', '1962']


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 3/5 ===
QUESTION: Which artist who had a major influence on the Gothic Revival is represented in the V&A's British galleries?
RAW COT OUTPUT:
 Horace Walpole is a major influence on the Gothic Revival. So, the final answer is Horace Walpole.
EXTRACTED ANSWER: Horace Walpole is a major influence on the Gothic Revival. So, the final answer is Horace Walpole.
GOLD ANSWERS: ['Horace Walpole', 'Horace Walpole', 'Horace Walpole']


Both `max_new_tokens` (=64) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 4/5 ===
QUESTION: In 1890, who did the university decide to team up with?
RAW COT OUTPUT:
 The American Baptist Education Society was founded in 1890 by the American Baptist Education Society and a donation from oil magnate and philanthropist John D. Rockefeller on land donated by Marshall Field. So, the final answer is the American Baptist Education Society.
EXTRACTED ANSWER: The American Baptist Education Society was founded in 1890 by the American Baptist Education Society and a donation from oil magnate and philanthropist John D. Rockefeller on land donated by Marshall Field. So, the final answer is the American Baptist Education Society.
GOLD ANSWERS: ['several regional colleges and universities', 'Des Moines College, Kalamazoo College, Butler University, and Stetson University', 'Des Moines College, Kalamazoo College, Butler University, and Stetson University', 'Des Moines College, Kalamazoo College, Butler University, and Stetson University']

=== Example 5/5 ===
QU